In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
import MDAnalysis.transformations as trans

import numpy as np
import pandas as pd

from tqdm import tqdm

import nglview as nv
import ipywidgets as widgets

import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from kneed import KneeLocator

import scipy as stats
from scipy.stats import norm
from scipy.stats import chi2
from statistics import mean
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth

import conserved_atoms.main as ca


In [3]:
stride = 1
# u = mda.Universe('run.pdb', 'run_100ps.xtc')
u = mda.Universe('md.tpr', 'md_10ps.xtc')

In [4]:
# Unwrap protein
protein = u.select_atoms('protein')
not_protein = u.select_atoms('not protein')


transforms = [trans.unwrap(protein),
              trans.center_in_box(protein, wrap=True),
              trans.wrap(not_protein)]

u.trajectory.add_transformations(*transforms)

In [5]:
df = ca.calc_density(u, pocket_definition='sphzone 12 (resid 140 or resid 144)', target='NA', align_on='CA')

Aligning trajectory on sphzone 12 (resid 140 or resid 144) and name CA.
(Note: You might have to realign if you want to calculate other variables that depend on relative distances.)
Aligning...done
Extracting atoms...done
Calculating denstity...done


In [6]:
binding_pocket = u.select_atoms('(name CA and sphzone 12 (resid 140 or resid 144)) or (resid 140 or resid 144)')

view = nv.show_mdanalysis(binding_pocket)
view.add_licorice()

bp_center = binding_pocket.centroid()
view.shape.add_sphere(bp_center, # Location
                      matplotlib.colors.to_rgba('#A43820', alpha=.6),
                      0.6, # Radius 
                      f"Binding pocket centroid") # Label


view

NGLWidget(max_frame=1000)

In [7]:
df

,x,y,z,id,name,time,density
0,68.736557,48.905769,14.638134,29357,NA,1.0,2.790343e-05
1,76.088326,56.303871,22.186039,29374,NA,1.0,2.065111e-05
2,-1.081748,-21.694979,15.711457,29362,NA,2.0,9.746101e-08
3,137.241150,49.465969,20.947153,29356,NA,4.0,1.300597e-06
4,75.439507,62.373127,15.827538,29372,NA,5.0,2.211765e-05
...,...,...,...,...,...,...,...
694,68.000374,62.755486,11.507454,29361,NA,497.5,3.125760e-05
695,74.338150,56.624531,15.005670,29372,NA,497.5,2.776327e-05
696,74.502373,53.075504,20.007727,29372,NA,498.0,2.539451e-05
697,65.897942,52.450256,13.084449,29368,NA,498.5,3.283030e-05


In [8]:
fig = px.scatter_3d(df[df.index % stride == 0],
                    x='x',
                    y='y',
                    z='z',
                    symbol='name',
                    # color='density',
                    color_continuous_scale='Spectral_r')

fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.update_scenes(aspectmode='data')


In [9]:
df_clustered, df_summary = ca.cluster(df, u.trajectory.n_frames, element='Na', outlier_treshold=0.05)

In [10]:
df_clustered

,x,y,z,id,name,time,density,cluster_id,mahalanobis,p,occupancy
0,68.736557,48.905769,14.638134,29357,NA,1.0,2.790343e-05,-1,0.506679,0.776204,NaN
1,76.088326,56.303871,22.186039,29374,NA,1.0,2.065111e-05,-1,0.549061,0.759929,NaN
2,-1.081748,-21.694979,15.711457,29362,NA,2.0,9.746101e-08,-1,3.259194,0.196009,NaN
3,137.241150,49.465969,20.947153,29356,NA,4.0,1.300597e-06,-1,2.291902,0.317921,NaN
4,75.439507,62.373127,15.827538,29372,NA,5.0,2.211765e-05,-1,0.769450,0.680638,NaN
...,...,...,...,...,...,...,...,...,...,...,...
691,65.266838,57.071156,16.559345,29367,NA,489.0,3.770780e-05,0,1.463458,0.481076,0.25974
692,62.377460,55.334805,16.382929,29373,NA,489.0,3.377335e-05,0,1.912987,0.384238,0.25974
693,66.001167,56.267258,16.310522,29359,NA,489.5,3.804855e-05,0,1.688702,0.429836,0.25974
694,63.013008,54.908424,16.746769,29373,NA,489.5,3.450525e-05,0,1.696497,0.428164,0.25974


In [11]:
df_summary

,cluster_id,size,x,y,z,x_std,y_std,z_std,radius,radius_std,occupancy
0,-1,436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,260,65.041666,56.499179,16.482226,0.778056,0.666011,0.593402,2.748527,1.199867,0.25974


In [13]:
# fig = px.scatter_3d(df_clustered[df_clustered['cluster_id'] != -1],
fig = px.scatter_3d(df_clustered,
                    x='x',
                    y='y',
                    z='z',
                    color='cluster_id',
                    color_continuous_scale='Spectral_r')

fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.update_scenes(aspectmode='data')

In [14]:
protein = u.select_atoms('protein')
binding_pocket = u.select_atoms('(name CA and sphzone 12 (resid 140 or resid 144)) or (resid 140 or resid 144)')
binding_pocket_res = u.select_atoms('(resid ' + ' or resid '.join(str(x) for x in np.unique(binding_pocket.atoms.resids)) + ') and (not backbone or name CA)')

print('resid ' + ' or resid '.join(str(x) for x in np.unique(binding_pocket.atoms.resids)))

centroid_bp = binding_pocket.center_of_mass()

view = nv.show_mdanalysis(binding_pocket)

# # Occupancy cutoff
occ_cutoff = 0.1

# view.add_representation('licorice', selection=binding_pocket_res.atoms.ids)
view.add_representation('licorice', selection='resid 140 or resid 144')
# view.add_representation('surface', selection=binding_pocket_res.atoms.ids, color='#90AFC5', opacity=0.1)

for index, row in df_summary.iterrows():

    if row['occupancy'] > occ_cutoff:
        if row.name != -1:    
            view.shape.add_sphere(row[['x', 'y', 'z']], # Location
                                cm.coolwarm(row['occupancy']),
                                row['radius'], # Radius 
                                # 2.27, # Radius 
                                f"Occupancy: {row['occupancy']:.1%}") # Label

# # # Add full water sphere
# # view.shape.add_sphere(row[['x', 'y', 'z']], # Location
# #                     matplotlib.colors.to_rgba('#66A5AD', alpha=.6),
# #                     12, # Radius 
# #                     f"Occupancy: {row['occupancy']:.1%}") # Label

view

resid 63 or resid 64 or resid 65 or resid 66 or resid 92 or resid 93 or resid 94 or resid 95 or resid 96 or resid 119 or resid 135 or resid 136 or resid 137 or resid 138 or resid 139 or resid 140 or resid 141 or resid 142 or resid 143 or resid 144 or resid 145 or resid 146 or resid 147 or resid 148


NGLWidget(max_frame=1000)